In [1]:
import pandas as pd
import networkx as nx

# Let's start creating a subgraph using one of the three files

loading all the Answers to questions file

In [2]:
answers=pd.read_csv("/Users/simone/Desktop/sx-stackoverflow-a2q.txt",sep=" ",header=None,names=['Active_User', 'Passive_User', 'Date'])

In [3]:
answers.head()

,Active_User,Passive_User,Date
0,9,8,1217567877
1,1,1,1217573801
2,13,1,1217606247
3,17,1,1217617639
4,48,2,1217618182


In [4]:
answers["Date"]=pd.to_datetime(answers['Date'],unit='s')

In [5]:
answers.head()

,Active_User,Passive_User,Date
0,9,8,2008-08-01 05:17:57
1,1,1,2008-08-01 06:56:41
2,13,1,2008-08-01 15:57:27
3,17,1,2008-08-01 19:07:19
4,48,2,2008-08-01 19:16:22


Select only the most recent dates 

In [6]:
answers.tail()

,Active_User,Passive_User,Date
17823520,2773607,1048138,2016-03-06 12:16:21
17823521,6018278,1982354,2016-03-06 12:16:29
17823522,3187183,1404306,2016-03-06 12:17:30
17823523,6022341,1667278,2016-03-06 12:17:40
17823524,6018304,6024256,2016-03-06 12:18:13


In [7]:
Recent_answers=answers[(answers['Date'] >= '2016-01-01') & (answers["Active_User"] != answers["Passive_User"])]

In [8]:
Recent_answers.groupby(["Active_User","Passive_User"]).count()

Date
Active_User Passive_User      
13          45843            1
            411103           1
            633183           1
            761118           1
            1079641          1
...                        ...
6024128     2072724          1
6024148     1001994          1
6024168     5753950          1
6024228     6024201          1
6024236     5644064          1

[534761 rows x 1 columns]

In [9]:
(Recent_answers.groupby(["Active_User","Passive_User"]).count()).sort_values("Date")

Date
Active_User Passive_User      
13          45843            1
3960021     5507913          1
            4792564          1
            4517922          1
            4157666          1
...                        ...
1873365     1590389         12
217408      1404049         13
85785       2101226         13
771848      1096892         14
217408      2258442         19

[534761 rows x 1 columns]

In [10]:
Recent_answers.head()

,Active_User,Passive_User,Date
17226995,3877963,4606570,2016-01-01 00:00:01
17226996,3539722,2450821,2016-01-01 00:00:12
17226997,2502532,5248833,2016-01-01 00:00:45
17226998,3829407,5712246,2016-01-01 00:00:49
17226999,5199921,2130738,2016-01-01 00:00:54


In [11]:
Recent_answers.isna().sum().sum()

0

In [12]:
Recent_answers.count()

Active_User     548450
Passive_User    548450
Date            548450
dtype: int64

Creating the multi-graph using the networkx library

# time can be removed beetween the attributes? just keeping it in the key? (solo per ricordarcelo)

we decided to use a multigraph where 2 nodes can be linked by more edges, each edge is unique given its key( time)

In [13]:
g = nx.from_pandas_edgelist(Recent_answers, "Active_User", "Passive_User", edge_attr=True,edge_key="Date",create_using=nx.MultiDiGraph())

il grafico è strano 😅 però da quello che ho capito spesso capita cosi' quando abbiamo molti nodi, ho comunque controllato guardano le edges e i nodi e sembra che si comporti bene

Number of nodes

In [14]:
g.number_of_nodes()

318017

In [15]:
g.number_of_edges()

548450

list of all nodes

In [ ]:
list(g.nodes)

list of all edges

In [ ]:
list(g.edges)

list of tuples of node and its adjacents

In [ ]:
[(n, nbrdict) for n, nbrdict in g.adjacency()]

# Now we can start creating the needed graph

In [19]:
Recent_answers=answers[(answers['Date'] >= '2016-01-01') & (answers["Active_User"] != answers["Passive_User"])].copy()

In [20]:
Recent_answers.count()

Active_User     548450
Passive_User    548450
Date            548450
dtype: int64

In [21]:
Recent_answers.isna().sum().sum()

0

I insert a column with the weight for each edge

In [22]:
Recent_answers["weight"]=1

In [23]:
Recent_answers.head()

,Active_User,Passive_User,Date,weight
17226995,3877963,4606570,2016-01-01 00:00:01,1
17226996,3539722,2450821,2016-01-01 00:00:12,1
17226997,2502532,5248833,2016-01-01 00:00:45,1
17226998,3829407,5712246,2016-01-01 00:00:49,1
17226999,5199921,2130738,2016-01-01 00:00:54,1


In [24]:
Recent_answers.groupby(["Active_User","Passive_User"]).count()

Date  weight
Active_User Passive_User              
13          45843            1       1
            411103           1       1
            633183           1       1
            761118           1       1
            1079641          1       1
...                        ...     ...
6024128     2072724          1       1
6024148     1001994          1       1
6024168     5753950          1       1
6024228     6024201          1       1
6024236     5644064          1       1

[534761 rows x 2 columns]

In [57]:
g = nx.from_pandas_edgelist(Recent_answers, "Active_User", "Passive_User", edge_attr=True,edge_key="Date",create_using=nx.MultiDiGraph())

# loading the comments on questions

In [26]:
comments_onQuestions=pd.read_csv("/Users/simone/Desktop/sx-stackoverflow-c2q.txt",sep=" ",header=None,names=['Active_User', 'Passive_User', 'Date'])

In [27]:
comments_onQuestions.head()

,Active_User,Passive_User,Date
0,4550,4550,1220729190
1,242,184,1220733503
2,4213,4946,1220768149
3,91,91,1220768295
4,2658,1874,1220771891


In [28]:
comments_onQuestions.count()

Active_User     20268151
Passive_User    20268151
Date            20268151
dtype: int64

In [29]:
comments_onQuestions["Date"]=pd.to_datetime(comments_onQuestions['Date'],unit='s')

In [30]:
comments_onQuestions.head()

,Active_User,Passive_User,Date
0,4550,4550,2008-09-06 19:26:30
1,242,184,2008-09-06 20:38:23
2,4213,4946,2008-09-07 06:15:49
3,91,91,2008-09-07 06:18:15
4,2658,1874,2008-09-07 07:18:11


Selecting only the most recent comments

In [31]:
Recent_comments_onQuestions=comments_onQuestions[(comments_onQuestions['Date'] >= '2016-01-01') & (comments_onQuestions["Active_User"] != comments_onQuestions["Passive_User"])].copy()

In [32]:
Recent_answers.isna().sum().sum()

0

we insert a column to add the weight to each edge

In [33]:
Recent_comments_onQuestions["weight"]=0.4

In [34]:
Recent_comments_onQuestions.head()

,Active_User,Passive_User,Date,weight
19237677,1569536,4854060,2016-01-01 00:00:13,0.4
19237679,1126841,3002444,2016-01-01 00:00:20,0.4
19237681,359284,1050303,2016-01-01 00:00:33,0.4
19237682,2189127,4022511,2016-01-01 00:00:58,0.4
19237685,1676363,502463,2016-01-01 00:01:03,0.4


In [35]:
Recent_comments_onQuestions.count()

Active_User     687560
Passive_User    687560
Date            687560
weight          687560
dtype: int64

In [36]:
g2 = nx.from_pandas_edgelist(Recent_comments_onQuestions, "Active_User", "Passive_User", edge_attr=True,edge_key="Date",create_using=nx.MultiGraph())

In [ ]:
[(n, nbrdict) for n, nbrdict in g.adjacency()]

In [38]:
Recent_answers.isna().sum().sum()

0

# combining the multi-graph using the networkx library

In [39]:
G = nx.compose(g,g2)

In [ ]:
[(n, nbrdict) for n, nbrdict in g.adjacency()]

loading all the comments on answers

In [41]:
comments_onAnswers=pd.read_csv("/Users/simone/Desktop/sx-stackoverflow-c2a.txt",sep=" ",header=None,names=['Active_User', 'Passive_User', 'Date'])

In [42]:
comments_onAnswers.head()

,Active_User,Passive_User,Date
0,1,91,1220713630
1,3,91,1220713792
2,380,350,1220715736
3,4642,2257,1220734307
4,4642,1324220,1220735746


In [43]:
comments_onAnswers["Date"]=pd.to_datetime(comments_onAnswers['Date'],unit='s')

In [44]:
comments_onAnswers.head()

,Active_User,Passive_User,Date
0,1,91,2008-09-06 15:07:10
1,3,91,2008-09-06 15:09:52
2,380,350,2008-09-06 15:42:16
3,4642,2257,2008-09-06 20:51:47
4,4642,1324220,2008-09-06 21:15:46


Selecting only the most recent comments

In [45]:
Recent_comments_onAnswers=comments_onAnswers[(comments_onAnswers['Date'] >= '2016-01-01') & (comments_onAnswers["Active_User"] != comments_onAnswers["Passive_User"])].copy()

In [46]:
Recent_comments_onAnswers.head()

,Active_User,Passive_User,Date
24495604,1104292,5149448,2016-01-01 00:00:25
24495605,137507,2102831,2016-01-01 00:00:26
24495607,2334092,2658050,2016-01-01 00:00:48
24495610,3405980,854083,2016-01-01 00:01:00
24495611,2334092,2658050,2016-01-01 00:01:10


In [47]:
Recent_comments_onAnswers["weight"]=0.6

In [48]:
Recent_comments_onAnswers.head()

,Active_User,Passive_User,Date,weight
24495604,1104292,5149448,2016-01-01 00:00:25,0.6
24495605,137507,2102831,2016-01-01 00:00:26,0.6
24495607,2334092,2658050,2016-01-01 00:00:48,0.6
24495610,3405980,854083,2016-01-01 00:01:00,0.6
24495611,2334092,2658050,2016-01-01 00:01:10,0.6


In [49]:
Recent_comments_onAnswers.count()

Active_User     618027
Passive_User    618027
Date            618027
weight          618027
dtype: int64

In [50]:
g3 = nx.from_pandas_edgelist(Recent_comments_onAnswers, "Active_User", "Passive_User", edge_attr=True,edge_key="Date",create_using=nx.MultiGraph())

combining the multi-graph using the networkx library

In [51]:
final_G = nx.compose(g3,G)

In [52]:
import pickle

In [53]:
pickle.dump(final_G, open('final_G', 'wb'))

In [55]:
# Load graph
with open("/Users/simone/Desktop/final_G", 'rb') as f: 
    G_loaded = pickle.load(f)

In [56]:
pickle.dump(g3, open('Recent_comments_onAnswers', 'wb'))

In [58]:
pickle.dump(g, open('Recent_answers', 'wb'))

In [59]:
pickle.dump(g2, open('Recent_comments_onQuestions', 'wb'))